In [67]:
#Due to compatability issues with spacy and numpy, we need to downgrade numpy to 1.23.5 and reinstall spacy
%pip uninstall numpy spacy thinc blis -y 

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: spacy 3.0.9
Uninstalling spacy-3.0.9:
  Successfully uninstalled spacy-3.0.9
Found existing installation: thinc 8.0.17
Uninstalling thinc-8.0.17:
  Successfully uninstalled thinc-8.0.17
Found existing installation: blis 0.7.11
Uninstalling blis-0.7.11:
  Successfully uninstalled blis-0.7.11
Note: you may need to restart the kernel to use updated packages.


In [68]:
%pip install -r requirements.txt

  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached spacy-3.8.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.3.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached spacy-3.0.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached blis-0.7.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached spacy-3.0.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
Using cached blis-0.7.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.2 MB)
Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
No

In [69]:
import re
import time
import itertools
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import boto3
import dill as pickle
import pickle

import nltk
from nltk.tokenize import WordPunctTokenizer
nltk.download('stopwords')

import spacy
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    train_test_split, GridSearchCV
)
from sklearn.metrics import confusion_matrix, classification_report

from xgboost.sklearn import XGBClassifier

# Load spacy model
nlp = spacy.load('en_core_web_sm')

# Suppress warnings
warnings.filterwarnings('ignore')

# AWS S3 client
s3 = boto3.client('s3')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/raksaurum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
!python -m spacy download en_core_web_sm

2025-04-15 01:35:33.314798: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 01:35:33.318718: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 01:35:33.327636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 01:35:33.341714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 01:35:33.345320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 01:35:33.356013: I tensorflow/core/platform/cpu_feature_gu

In [71]:
def custom_tokenizer(doc):
    tokens = nlp(doc)
    return [token.lemma_ for token in tokens if not token.is_punct]

In [72]:
def create_upload(object, file_name):
    session = datetime.now().strftime('%Y%m%d_%H%M%S')
    full_name = '{}_{}.pkl'.format(file_name, session)
    with open(full_name, 'wb') as file:
        pickle.dump(object, file)

In [73]:
df= pd.read_csv("training.1600000.processed.noemoticon.csv",encoding="latin-1",names=['target','id','date','flag','user','text'])
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [74]:
def getCleanTweet(text):
    tok = WordPunctTokenizer()
    user_pattern = '@[A-Za-z0-9_]+'
    http_pattern = 'https?://[^ ]+'
    www_pattern = 'www.[^ ]+'
    combined_pattern = '|'.join((user_pattern, http_pattern, www_pattern))
    negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                    "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                    "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                    "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                    "mustn't":"must not"}
    neg_pattern = re.compile('\b(' + '|'.join(negations_dic.keys()) + ')\b')
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pattern, '', bom_removed)
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], stripped)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled).lower()
    cleaned = (" ".join(x for x in tok.tokenize(letters_only) if len(x) > 1)).strip()
    cleaned = ''.join(k + k if sum(1 for i in g) > 1 else k for k, g in itertools.groupby(cleaned))
    return cleaned

In [75]:
print(df.head())
print(df['text'].apply(type).value_counts())
def getCleanTweet(text):
    return text.strip() if isinstance(text, str) else None
print(df.columns)

   target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  
text
<class 'str'>    1600000
Name: count, dtype: int64
Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')


In [76]:
df['clean_text'] = [getCleanTweet(text) for text in df['text']]
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   target      1600000 non-null  int64 
 1   id          1600000 non-null  int64 
 2   date        1600000 non-null  object
 3   flag        1600000 non-null  object
 4   user        1600000 non-null  object
 5   text        1600000 non-null  object
 6   clean_text  1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [77]:
# Down Sample
tweets_subsampled_1, tweets_subsampled_2 = train_test_split(df, test_size=0.1)

In [78]:
# Split between outcome and Features
y = tweets_subsampled_2['target']
X = tweets_subsampled_2['text']

In [79]:
start_time = time.time()

# Create lemmatizer using spacy
lemmatizer = spacy.lang.en.English()

In [80]:
pipe = Pipeline(steps=[('vectidf', TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',
                                                   lowercase=True, use_idf=True, max_df=0.5,
                                                   min_df=2, norm='l2', smooth_idf=True)),
                       ('svd', TruncatedSVD(500)),
                       #('norm',Normalizer(copy=False))
                       ])

In [81]:
tweets_transform = pipe.fit_transform(X)

In [82]:
print("Transform Data - Execution time: %s seconds ---" % (time.time() - start_time))

# splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tweets_transform, y, test_size=0.25)

Transform Data - Execution time: 662.9913065433502 seconds ---


In [83]:
# Base Model Build
warnings.filterwarnings('ignore')
start_time = time.time()

In [84]:
xgb_model = XGBClassifier(random_state=10)

In [85]:
parameters = {'n_jobs': [-1],
              }

In [86]:
# Map target variable to binary classes
y_train_binary = y_train.map(lambda x: 1 if x == 4 else 0)
y_test_binary = y_test.map(lambda x: 1 if x == 4 else 0)

# Update GridSearchCV and fit the model
clf = GridSearchCV(xgb_model, parameters, cv=3, verbose=0, n_jobs=1)
clf.fit(X_train, y_train_binary)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=1, param_grid={'n_jobs': [-1]})

In [87]:
print("Base Line Model - Execution time: %s seconds ---" % (time.time() - start_time))
print("Base Line Model - CV Score: " + str(clf.best_score_))
print("Best Params: " + str(clf.best_params_))

Base Line Model - Execution time: 94.29208588600159 seconds ---
Base Line Model - CV Score: 0.7269249999999999
Best Params: {'n_jobs': -1}


In [88]:
warnings.filterwarnings('ignore')
start_time = time.time()
print("Starting full model training...")

Starting full model training...


In [89]:
tweets_subsampled_1, tweets_subsampled_2 = train_test_split(df, test_size=0.5)

In [90]:
y = tweets_subsampled_2['target']
X = tweets_subsampled_2['text']

In [91]:
# Transform Data
print("Starting vectorization...")
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',
                             lowercase=True, use_idf=True, max_df=0.5,
                             min_df=2, norm='l2', smooth_idf=True, ngram_range=(1, 2))

tweets_tfidf = vectorizer.fit_transform(X)

print("Vectorizing Finished. Number of features: %d" % tweets_tfidf.get_shape()[1])

Starting vectorization...
Vectorizing Finished. Number of features: 534955


In [92]:
print("Starting Dimension Reduction...")

Starting Dimension Reduction...


In [93]:
print('Start model training...')
start_time = time.time()
X_train, X_test, y_train, y_test = train_test_split(tweets_tfidf, y, test_size=0.3)

Start model training...


In [100]:
# Filter only binary classes (0 and 1)
train_valid_indices = ((y_train == 0) | (y_train == 1)).to_numpy()
X_train = X_train[train_valid_indices]
y_train = y_train[train_valid_indices]

test_valid_indices = ((y_test == 0) | (y_test == 1)).to_numpy()
X_test = X_test[test_valid_indices]
y_test = y_test[test_valid_indices]


In [103]:
print("y_train unique values:", np.unique(y_train))
print("Length of y_train:", len(y_train))


y_train unique values: [0]
Length of y_train: 280006


In [105]:
xgb_model = XGBClassifier(base_score=0.5,
                          max_depth=5,
                          min_child_weight=5,
                          gamma=0.1,
                          subsample=0.8,
                          colsample_bytree=0.8,
                          scale_pos_weight=1,
                          random_state=10,
                          n_estimators=5000,
                          learning_rate=0.01,
                          n_jobs=-1)

In [106]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [107]:
print("Test Set Score: " + str(xgb_model.score(X_test, y_test)))
print("Train - Execution time: %s seconds ---" % (time.time() - start_time))

Test Set Score: 1.0
Train - Execution time: 1335.0030889511108 seconds ---


In [108]:
pred = xgb_model.predict(X_test)
pred = np.argmax(pred)
y_true = np.argmax(y_test)

In [109]:
y_pred = xgb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred))

[[119855]]


In [110]:
print (classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    119855

    accuracy                           1.00    119855
   macro avg       1.00      1.00      1.00    119855
weighted avg       1.00      1.00      1.00    119855

